# Fast Style Transfer
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/johnson-fast-style-transfer/blob/main/fast-style-transfer.ipynb)

This notebook lets you try out image stylization using the approach described
by Johnson, Alahi, and Fei-Fei in their article _Perceptual Losses for
Real-Time Style Transfer and Super-Resolution_.  For more details, see my
github repositories [`johnson-fast-style-transfer`](
https://github.com/mdehling/johnson-fast-style-transfer) and
[`nstesia`](https://github.com/mdehling/nstesia).

In [ ]:
# At the time of writing, the versions of pip and setuptools provided by colab
# do not have full `pyproject.toml` support --- they must be updated before
# installing the nstesia package.  This cell will do just that.
#
# Once this cell has finished running, YOU MUST RESTART THE COLAB RUNTIME.
try:
    from google import colab

    # Pull everything else from the repository in to the Colab environment.
    !git init .
    !git remote add origin https://github.com/mdehling/johnson-fast-style-transfer.git
    !git pull --depth=1 origin main

    # These are just to avoid some scary-looking (but harmless) error messages.
    !pip uninstall -q -y numba
    !pip install -q 'jedi>=0.10'

    # Minimum versions required for PEP-660 support.
    !pip install -q 'pip>=21.3' 'setuptools>=64'

    # Finally install the one we came for.
    !pip install -q -r requirements.txt

except ImportError:
    pass

In [ ]:
from IPython.display import display

from os import environ as env
env['TF_CPP_MIN_LOG_LEVEL'] = '2'               # hide info & warnings
env['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'       # grow GPU memory as needed

import tensorflow as tf
import nstesia as nst

## Downloading the Models

In [ ]:
# This will download and unpack the saved models.
![ ! -e saved ] && \
    gdown 1zCIvHLiFPUyf3IdKk3Jwt09gtUoSRRIF && \
    tar xjf saved.tar.bz2 && \
    rm -f saved.tar.bz2

## Simple Image Stylization

In [ ]:
def show_image(image_tensor):
    display(tf.keras.utils.array_to_img(tf.squeeze(image_tensor,axis=0)))

In [ ]:
model = nst.johnson_2016.StyleTransferModel.from_saved('saved/delaunay')
content_image = nst.io.load_image('img/content/stanford.jpg')

pastiche_image = model(content_image)
show_image(pastiche_image)

## Instance vs Batch Normalization

In [ ]:
def load_style_models(styles):
    """
    Load style transfer models from the `saved` directory.

    Args:
        styles:
            A list of strings.  The names of directories under `saved/`
            containing the desired saved models in 'tf' format.

    Returns:
        A list of `nstesia.johnson_2016.StyleTransferModel` instances.
    """
    return [
        nst.johnson_2016.StyleTransferModel.from_saved(f'saved/{style}')
        for style in styles
    ]

In [ ]:
# img/results/instance-vs-batch-norm.png
content_image = nst.io.load_image('img/content/chicago.jpg')
style_models = load_style_models(['candy', 'candy-bn'])

grid_image = nst.image.grid([model(content_image) for model in style_models])
show_image(grid_image)

## Content-Style Grids

In [ ]:
def content_style_grid(image_files, styles):
    """
    Show a grid of stylizations with content images as rows, styles as columns.

    Args:
        images_files:
            A list of strings.  The file names of the content images to load.
        styles:
            A list of strings.  The names of styles to be loaded from disk.

    Returns:
        An image representing the grid of stylizations.
    """
    content_images = [ nst.io.load_image(file) for file in image_files ]
    style_models = load_style_models(styles)

    images = [None] + [model.style_image for model in style_models]
    for content_image in content_images:
        images += [content_image] + [
            model(content_image) for model in style_models
        ]

    return nst.image.grid(images, ncols=len(styles)+1)

In [ ]:
# img/results/content-style-matrix-1.png
grid_image = content_style_grid(
    ['img/content/sunflower.jpg', 'img/content/bochum.jpg'],
    styles=['candy', 'cubism', 'udnie'],
)
show_image(grid_image)

In [ ]:
# img/results/content-style-matrix-2.png
grid_image = content_style_grid(
    ['img/content/brad.jpg', 'img/content/karya.jpg'],
    styles=['bathing', 'delaunay', 'scream'],
)
show_image(grid_image)